## Using a Pretrained Network: Feature Extraction

Convnets used for image classification comprise of two parts: a series of convolutional and pooling layers followed by a densely connected classifier layer. The first part is called the _convolutional base_ of the network. In the context of convolutional networks, _feature extraction_ using a pre-trained network consists of using the convolutional base of this pre-trained network and passing new data through it and then feeding its output to a new, as yet untrained, convnet. 

In our case, we will pass the images of cats and dogs through the convolutional base of the VGG16 Network and then use its output as input for our own network that is specifically being trained to distinguish dogs from cats. One can think of this process as training the new convnet to learn the image representations of the pre-trained network. The classifier of the pre-trained network is specific to the classification problem at hand. Furthermore, the dense classifier totally ignores the spatial position of features such as edges, colours and textures. In the case of VGG16, its classifier is trained on 1000 different classes. The weights information of the classifier itself is not very useful while training a new model to distinguish dogs from cats. 

## Instantiating the VGG16 Convolutional Base

In [1]:
from keras.applications import VGG16

# The include_top argument specifies whether we want to include the densely connected classifier in the top
conv_base = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(150, 150, 3))

In [2]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## Imports, Directory Paths

In [3]:
import os
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers


base_dir = './'

train_dir = os.path.join(base_dir, 'train') 
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')                           

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')                 
                                             
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs') 

## Extracting Features from a Pre-trained Convnet

In [4]:
datagen = ImageDataGenerator(rescale=1./255)


def extract_features(directory, sample_count, batch_size=20):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i = 0
    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        
        i += 1
        
        if i * batch_size >= sample_count:
            break                                                         
    
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
